In [2]:
from pyGROMACS import GROMACS_setup

# pyGROMACS

This notebook enables the setup and analysis of MD simulations using GROMACS in an automated fashion. The idea is to introduce the **FAIR** principles:

1. **F**indable ( metadata and data should be easy to find for humans and computers )
2. **A**ccessible ( (meta)data should accessable by standardised communications protocol, e.g.: file formats )
3. **I**nteroperable ( data need to interoperate with applications or workflows for analysis, storage, and processing )
4. **R**eusable ( (meta)data should be well-described to replicate it in different settings )

This notebook uses pyGROMACS, GROMACS, and moleculegraph to write topology, itp, and gro files.

In [3]:
gromacs_setup = GROMACS_setup( system_setup = "input/system.yaml", 
                               simulation_default = "input/defaults.yaml",
                               simulation_ensemble = "input/ensemble.yaml",
                               submission_command = "qsub"
                             )

## Build topology, itp and gro files using moleculegraph

This will create a folder called "topology" in the system folder specified in the system setup

In [4]:
gromacs_setup.write_topology(  )


Utilize moleculegraph to generate itp, topology and initial gro files of every molecule in the system!

Done! Topology paths and molecule coordinates are added within the class.



## Setting up a simulation pipeline

In this section the possibility to setup a simulation folder, along with a simulation pipeline using several ensembles is set up.

<p align="center">
    <img src="pipeline_thermo.png" alt="Pipeline Image" width="1000"/>
</p>

1) Setup simulation and build initial system (if not provided)
2) Submit jobs to cluster

In [5]:
# Define the simulation folder
simulation_folder = "md_thermo"

# Define the ensembles that should be simulated (definition what each ensemble means is provided in yaml file)
ensembles = [ "em", "nvt", "npt_equilibration", "npt_production" ]

# Define the simulation time per ensemble in nano seconds (for em provide number of iterations)
simulation_times = [ 10000, 1.0, 20.0, 50.0 ]

# Define initial systems, in case the simulation should be continued from a prior simulation.
# Therefore, provide one initial structure for each temperature & pressure state.
# If none are specified, gmx-insert will create them.
initial_systems = []

# Provide kwargs that should be passed into the mdp template directly
mdp_kwargs = {}

# Define number of copies
copies = 2

# Define if the inital system should build locally or with the cluster
on_cluster = False


gromacs_setup.prepare_simulation( folder_name = simulation_folder, ensembles = ensembles, 
                                  simulation_times = simulation_times, initial_systems = initial_systems, 
                                  mdp_kwargs = mdp_kwargs, copies = copies, on_cluster = on_cluster 
                                )

State: T = 298.1 K, P = 1.0 bar

Building system based on provided molecule numbers and coordinate files!


Build system locally! Wait until it is finished.

Build successful



In [6]:
# Submit the simulations
gromacs_setup.submit_simulation()


Submitting simulations at Temperature = 298.1 K, Pressure = 1.0 bar

Submitting job: /home/st/st_st/st_ac137577/workspace/software/pyGROMACS/example/liquid/water/md_thermo/temp_298.1_pres_1.0/copy_0/job_298.sh
11455384


Submitting job: /home/st/st_st/st_ac137577/workspace/software/pyGROMACS/example/liquid/water/md_thermo/temp_298.1_pres_1.0/copy_1/job_298.sh
11455385


Submitting job: /home/st/st_st/st_ac137577/workspace/software/pyGROMACS/example/liquid/water/md_thermo/temp_298.1_pres_1.0/copy_2/job_298.sh
11455386




## Data analysis

This generic tool allows to extract properties using GROMACS commands for an specified simulation folder and ensemble. This will average the properties over each copy and safe the produced data in each temperature & pressure folder as json file

### Extract and analysis of generic properties

In [7]:
# Extract properties from GROMACS and analyse them

# Define analysis folder
analysis_folder = "md_thermo"

# Define analysis ensemble
ensemble = "03_npt_production"

# Define GROMACS command
command = "energy"

# Properties to extract
properties = ["Density", "Volume", "Enthalpy"]

# Special args to gromacs analysis
args = [ ]

# XVG output name
output_name = "properties" 

# Submit analysis on cluster
on_cluster = False

# Perform extraction from gromacs
extract = True

gromacs_setup.analysis_extract_properties( analysis_folder = analysis_folder, ensemble = ensemble, extracted_properties = properties, fraction = 0.25,
                                           command = command, args = args, output_name = output_name, on_cluster = on_cluster, extract = extract )

Extract locally

/home/st/st_st/st_ac137577/workspace/software/pyGROMACS/example/liquid/water/md_thermo/temp_298.1_pres_1.0/copy_0/03_npt_production/extract_properties.sh
/home/st/st_st/st_ac137577/workspace/software/pyGROMACS/example/liquid/water/md_thermo/temp_298.1_pres_1.0/copy_1/03_npt_production/extract_properties.sh
/home/st/st_st/st_ac137577/workspace/software/pyGROMACS/example/liquid/water/md_thermo/temp_298.1_pres_1.0/copy_2/03_npt_production/extract_properties.sh 

Wait until extraction is done...


/home/st/st_st/st_ac137577/miniconda3/envs/py311/lib/python3.11/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/home/st/st_st/st_ac137577/miniconda3/envs/py311/lib/python3.11/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Extraction finished!


Temperature: 298.15, Pressure: 1.0
   

Averaged values over all copies:

    property          mean         std    unit
0    Volume     29.063974    0.130056    nm^3
1   Density   1029.334041    4.604472  kg/m^3
2  Enthalpy -29696.231436  161.909629  kJ/mol 

